In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import sagemaker
import boto3
from io import BytesIO

module_path = os.path.abspath(os.path.join('../py-conjugated/networks/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import morphology_networks as net
import model_training as train
import model_testing as test
import physically_informed_loss_functions as PhysLoss
import network_utils as nuts

torch.manual_seed(28)

In [3]:
bucket_name = 'sagemaker-us-east-2-362637960691'
train_data_location = 'py-conjugated/m2py_labels/OPV_labels/train_set/'
test_data_location = 'py-conjugated/m2py_labels/OPV_labels/test_set/'
model_states_location = 'py-conjugated/model_states/OPV/'

client = boto3.client('s3')
resource = boto3.resource('s3')
s3_bucket = resource.Bucket(bucket_name)

NoCredentialsError: Unable to locate credentials

In [ ]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/OPVCNN/'

role = sagemaker.get_execution_role()

In [ ]:
train_data = nuts.OPV_ImDataset(s3_bucket, train_data_location)
test_data = nuts.OPV_ImDataset(s3_bucket, test_data_location)

train_data_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = len(train_data), shuffle = True)
teest_data_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = len(test_data), shuffle = True)

In [ ]:
# Hyper parameters
num_epochs = 10
im_learning_rate = 0.008

# Device configuration (GPU if available, otherwise CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
im_branch_model = net.OPV_m2py_NN(2)

#define the loss function and the optimizer
im_criterion = nn.CrossEntropyLoss()
im_optimizer = torch.optim.Adam(im_branch_model.parameters(), lr = im_learning_rate)


In [ ]:
%autoreload

im_train_epoch_losses = []
im_test_epoch_losses = []

for epoch in range(num_epochs):
    
    # First Train the image branch
    im_train_epoch_loss = train.train_OPV_m2py_model(model = im_branch_model,
                                   training_data_set = im_training_data_set,
                                   criterion = im_criterion,
                                   optimizer = im_optimizer)
    
    im_train_epoch_losses.append(im_train_epoch_loss)
    
    im_test_epoch_loss = test.eval_OPV_m2py_model(model = im_branch_model,
                                 testing_data_set = im_testing_data_set,
                                 criterion = im_criterion)
    
    im_test_epoch_losses.append(im_test_epoch_loss)

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

epochs = np.arange(1, (num_epochs+1), 1)

plt.plot(epochs, im_train_epoch_losses, c = 'k', label = 'training error')
plt.plot(epochs, im_test_epoch_losses, c = 'r', label = 'testing error')
plt.legend(loc = 'upper right')
plt.show()